In [1]:
#setting libraries
import os
import re
import datetime

In [2]:
#reading data and separating data into test and bench files
tests_files=[a for a in os.listdir('./data.nogit/') if 'test' in a]
bench_files=[a for a in os.listdir('./data.nogit/') if 'bench' in a]
# os.getcwd()

In [ ]:
#overview of the test files
bits={} #key:file name value:number of bits

for a in tests_files:
    total_bits=0
    n_input=0
    n_outpus=0
    with open(f'./data.nogit/{a}','r') as file:
        while True:
            line=file.readline().strip()
            if not(line):
                break
            else: #there's available data to read
                total_bits=max(total_bits,len(line))
                n_outpus=line.count('-')
                n_input=len(line)-n_outpus
    bits[a]={"total_bits":total_bits,"INPUT":n_input,"OUTPUT":n_outpus}
    file.close()

for a,b in bits.items():
    print(f'file: {a}, total bits to work: {b}')

In [ ]:
def extract_number_from_string(string:str):
    '''
    :param str string: the string of values to evaluate
    input string to extract the integer value
    '''
    print(string)
    aux=re.findall(r'(\d+) \w+',string)
    print(aux)
    return int(aux[0])
#overview of bench files

n_vales={} #key:file_name value:{key: input/output/gates value:read value from file}
exp_values={} #key: input/output/gates value:expected number
for a in bench_files:
    n_vales[a]={}
    exp_values[a]={}
    aux=0
    print(f'read file: {a}')
    with open(f'./data.nogit/{a}','r') as file:
        while True:
            line=file.readline().strip()
            if aux>3:
                break
            if not(line):
                aux+=1           
                continue
            else:
                if line[0]=='#': #it show the information about the file
                    try:
                        exp_values[a][line.split(' ')[2].upper()]=extract_number_from_string(line.lower())
                    except:
                        continue
                else:
                    gate_str="".join(filter(str.isalpha,line))
                    if gate_str not in n_vales[a]:
                        n_vales[a][gate_str]=0
                    n_vales[a][gate_str]+=1
    file.close()


In [ ]:
gates=[] #getting the names of the gates used in the bench files
for a,b in n_vales.items():
    print (a,b)
    for c  in b.keys():
        if c not in ['INPUT','OUTPUT']:
            if c not in gates:
                gates.append(c)

print(gates)

Code for running tests
Testing will be done with file c17.tests and c17.bench

In [3]:
#setting time to Japan time (UTC+9)
jpTime=datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
#when testing started
t0=jpTime.now()

In [4]:
def log(msg:str):
    '''
    :param str msg: the message to appear in the log
    '''
    delta=jpTime.now()-t0 #show the running time
    print(f'{delta}...{msg}')

In [14]:
#logic gate tryout
def logic_gate(gate:str, inputs:list):
    '''
    defining the logic gate functioning

    Params:
        gate(str): 'NAND', 'AND', 'OR', 'NOT', 'NOR', 'BUFF', 'XOR' name of gates in bench files
        inputs(list): defines the inputs for the gate [A,B,C...], the number of elements in the function will determine the amount of pins for the logic gate.EXCEPT for NOT gates it will take ONLY the first element

    Returns:
        (bool): a single logic value to assign to the corresponding 
    '''
    if gate=='NAND':
        holder=1
        [holder:=holder&x for x in inputs]
        return 0 if holder else 1
    elif gate=="AND":
        holder=1
        [holder:=holder&x for x in inputs]
        return 1 if holder else 0
    elif gate=="OR":
        holder =0
        [holder:=holder|x for x in inputs]
        return 1 if holder else 0
    elif gate=="NOT":
        return 0 if inputs[0] else 1
    elif gate=="NOR":
        holder =0
        [holder:=holder|x for x in inputs]
        return 0 if holder else 1
    elif gate=="BUFF":
        return 1 if inputs[0] else 0
    elif gate=="XOR":
        holder=0
        [holder:=holder^x for x in inputs]
        return 0 if holder else 1

In [ ]:
#reading the bench file to assign the input, output and intermediate connections.
#read only once (the rutine is the same for ALL the data)
def read_bench(bench_file:str):
    aux=0
    signal_line={} #key:singal line value: logic value (0/1)
    primary_input=[]
    primary_output=[]
    circ_struc=[] #circuit structure
    with open(bench_file) as f:
        while True:
            a=f.readline().strip()
            if aux>3:
                break
            if not(a):
                aux+=1
                continue
            else:
                if "#" in a:
                    continue
                else:
                    if 'INPUT' in a or 'OUTPUT' in a: #connections for primary input and primary output
                        b=''.join(filter(str.isnumeric,a))
                        signal_line[b]=''
                        if 'INPUT' in a:
                            primary_input.append(b)
                        else:
                            primary_output.append(b)
                    else: # signal = gate (input signal)
                        signal_line[a.split(' = ')[0]]=''
                        circ_struc.append(a)
    f.close()
    log (f"Finished reading bench file: {bench_file}")
    return signal_line, circ_struc, primary_input, primary_output

In [21]:
#reading the test file for data and testing
#this script produces the "correct" behaviour
def read_test(test_file,signal_line,circ_struc,primary_input,primary_output):
    with open(test_file) as f:
        while True:
            line=f.readline().strip()
            if not(line):
                break
            else: #I assume the data is organized in the bench stated order
                #assigning the test data to the corresponding signals
                for a in range(len(primary_input)):
                    try:
                        signal_line[primary_input[a]]=int(line[a])
                    except:
                        continue
                
                #executing the testing from the bench file (running the circuit)
                #structure= output signal= logic gate (input signal)
                for a in circ_struc:
                    output_signal=a.split(' = ')[0]
                    gate=a.split(' = ')[1].split('(')[0]
                    input_signal_index=a.split(' = ')[1].split('(')[1][:-1]
                    input_signals=[signal_line[x.strip()] for x in input_signal_index.split(',')]
                    # print(a,gate,input_signals)
                    signal_line[output_signal]=logic_gate(gate,input_signals)
                circuit_output=[signal_line[x] for x in primary_output]
                # print(f'Data input: {line[:line.index('-')]}, \tData output: {circuit_output}')
    f.close()
    log(f"Finished running test file {test_file}")
    return circuit_output

In [22]:
for a in bench_files:
    print(a)
    b,c,d,e=read_bench(f'./data.nogit/{a}')
    print(read_test(f'./data.nogit/{a[:a.index('.')]}.tests',b,c,d,e))

c880.bench
0:10:37.165625...Finished reading bench file


0:10:37.229076...Finished running test file ./data.nogit/c880.tests
[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
c432.bench
0:10:37.229442...Finished reading bench file
0:10:37.254885...Finished running test file ./data.nogit/c432.tests
[1, 1, 1, 0, 0, 0, 0]
c1355.bench
0:10:37.255269...Finished reading bench file
0:10:37.354122...Finished running test file ./data.nogit/c1355.tests
[1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0]
c7552.bench
0:10:37.356191...Finished reading bench file
0:10:37.903156...Finished running test file ./data.nogit/c7552.tests
[0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0]
c17.bench
0:10:37.903590...Finish